In [1]:
import datetime
import pandas as pd
import numpy as np

In [2]:
migrations = pd.read_csv("metro_migrations.csv")
net_flow = pd.read_csv("migration_historical.csv")
migrations.head(5)

,Date,Origin,OriginDisplay,OriginState,Destination,DestDisplay,DestState,Pct Of Origin Looking At Dest,Pct Of Dest Looking From Origin,Dest As Pct Of Origin Leavers,Origin As Pct Of Dest Incomers
0,2017/1/1,"Albany-Schenectady, NY","Albany, NY",NY,"Albany-Schenectady, NY","Albany, NY",NY,63.10%,61.20%,NaN,NaN
1,2017/1/1,"Albany-Schenectady, NY","Albany, NY",NY,"Boston-Worcester-Providence, MA-RI-NH-CT","Boston, MA",MA,6.10%,0.10%,16.70%,1.00%
2,2017/1/1,"Albany-Schenectady, NY","Albany, NY",NY,"New York-Newark, NY-NJ-CT-PA","New York, NY",NY,5.80%,0.10%,15.70%,1.50%
3,2017/1/1,"Albany-Schenectady, NY","Albany, NY",NY,"Washington-Baltimore-Arlington, DC-MD-VA-WV-PA","Washington, DC",DC,4.20%,0.00%,11.30%,0.40%
4,2017/1/1,"Albany-Schenectady, NY","Albany, NY",NY,"Seattle-Tacoma, WA","Seattle, WA",WA,2.00%,0.00%,5.40%,0.10%


In [3]:
net_flow.head(5)
# using this dataframe, we may analysis the trends of move in state level and we can select some of the most remarkable cases to visualize it 
# on the map: for instance, top 5 states which has the max net flow and top 5 min-flow states. Then we can try to analyze the reason using 
# other datasets, such as income, number of covid cases, etc.

# for smaller region, we can let the information to be shown when zoom in. In the big picture, it will not appear.

,date,Metro,State,Net flow
0,2017/1/1,Sacramento,CA,4089
1,2017/1/1,Phoenix,AZ,2744
2,2017/1/1,Las Vegas,NV,2476
3,2017/1/1,Dallas,TX,2071
4,2017/1/1,Portland,OR,1556


In [4]:
migrations['Date'] = pd.to_datetime(migrations['Date'])
mask = migrations['Date'].dt.year >= 2020
df_filter = migrations[mask]
print(df_filter.shape)
df_filter.head(5)

(36350, 11)


,Date,Origin,OriginDisplay,OriginState,Destination,DestDisplay,DestState,Pct Of Origin Looking At Dest,Pct Of Dest Looking From Origin,Dest As Pct Of Origin Leavers,Origin As Pct Of Dest Incomers
4988,2021-10-01,"Boise City-Mountain Home-Ontario, ID-OR","Boise City, ID",ID,"Boise City-Mountain Home-Ontario, ID-OR","Boise City, ID",ID,69.40%,39.30%,NaN,NaN
5138,2021-07-01,"Boise City-Mountain Home-Ontario, ID-OR","Boise City, ID",ID,"Boise City-Mountain Home-Ontario, ID-OR","Boise City, ID",ID,67.80%,39.70%,NaN,NaN
5754,2021-04-01,"Boise City-Mountain Home-Ontario, ID-OR","Boise City, ID",ID,"Boise City-Mountain Home-Ontario, ID-OR","Boise City, ID",ID,54.30%,27.10%,NaN,NaN
5802,2020-04-01,"Boise City-Mountain Home-Ontario, ID-OR","Boise City, ID",ID,"Boise City-Mountain Home-Ontario, ID-OR","Boise City, ID",ID,49.40%,18.50%,NaN,NaN
5913,2020-10-01,"Boise City-Mountain Home-Ontario, ID-OR","Boise City, ID",ID,"Boise City-Mountain Home-Ontario, ID-OR","Boise City, ID",ID,48.30%,17.80%,NaN,NaN


In [5]:
mask2 = df_filter['OriginDisplay'] == df_filter['DestDisplay']
df_filter = df_filter[~mask2]
# df_filter = df_filter.dropna()
print(df_filter.shape)
df_filter.isna().any()

(35336, 11)


Date                               False
Origin                             False
OriginDisplay                      False
OriginState                        False
Destination                        False
DestDisplay                        False
DestState                          False
Pct Of Origin Looking At Dest      False
Pct Of Dest Looking From Origin    False
Dest As Pct Of Origin Leavers      False
Origin As Pct Of Dest Incomers     False
dtype: bool

In [6]:
df_filter = df_filter[['Date', 'OriginDisplay', 'DestDisplay', 'Dest As Pct Of Origin Leavers', 'Origin As Pct Of Dest Incomers']]
df_filter

,Date,OriginDisplay,DestDisplay,Dest As Pct Of Origin Leavers,Origin As Pct Of Dest Incomers
7534,2020-07-01,"Tucson, AZ","Boise City, ID",4.40%,2.30%
7577,2020-04-01,"Tucson, AZ","Boise City, ID",4.50%,2.20%
7622,2020-07-01,"Corpus Christi, TX","Boise City, ID",3.30%,0.30%
7691,2020-10-01,"Tucson, AZ","Boise City, ID",3.20%,2.10%
7772,2021-01-01,"Tucson, AZ","Boise City, ID",3.00%,2.80%
...,...,...,...,...,...
61495,2022-01-01,"Appleton, WI","Los Angeles, CA",2.10%,0.00%
61496,2022-01-01,"Charlottesville, VA","Los Angeles, CA",1.60%,0.00%
61497,2022-01-01,"Gainesville, FL","Los Angeles, CA",2.50%,0.00%
61498,2022-01-01,"Pensacola, FL","Los Angeles, CA",1.90%,0.00%
